In [2]:
from lab.postprocessing.toolbox import load_model
model ,loader = load_model("/home/iailab36/aliakbar/rnn-cnn/scripts/results/cifar/resnet/resnet_RNN_dim_10_rec_2/","0718-121833",device='cpu')

Files already downloaded and verified


In [3]:
model_fixed ,loader_f = load_model("/home/iailab36/aliakbar/rnn-cnn/scripts/results/cifar/resnet/resnet18/","0819-175224",device='cpu')

Files already downloaded and verified


In [3]:
model_mlp ,loader_f2 = load_model("/home/iailab36/aliakbar/rnn-cnn/scripts/results/cifar/resnet/resnet18+mlp3/","0830-133113",device='cpu')

Files already downloaded and verified


In [4]:
print(sum(p.numel() for p in model_mlp.parameters()))    
print(sum(p.numel() for p in model.parameters()))  


11174182
11174292


In [4]:
## Dataloader For foolbox
import torch
all_imgs = []
all_imgs = torch.tensor(all_imgs,dtype=torch.float32).to("cpu")
all_labels = []
all_labels = torch.tensor(all_labels,dtype=torch.long).to("cpu")
all_lstacks = []
all_lstacks = torch.tensor(all_lstacks,dtype=torch.float32).to("cpu")
all_f_recs = []
all_f_recs = torch.tensor(all_f_recs,dtype=torch.float32).to("cpu")
for batch in loader.valid:
    labels_f = batch[1].to("cpu")
    input_f= batch[0].to("cpu")
    _ , f_recs = model(input_f,0)
    
#     all_lstacks=torch.cat((all_lstacks,lstacks),0)
    all_f_recs=torch.cat((all_f_recs,f_recs),0)
    all_imgs=torch.cat((all_imgs,input_f),0)
    all_labels=torch.cat((all_labels,labels_f),0) 

In [6]:
import torch

import torchvision.models as models
# model = models.resnet18(pretrained=True).eval().cuda()
# for batch in loader.valid:
#     labels = batch[1]
#     input= batch[0]
#     break
torch.set_grad_enabled(True)
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import FGSM
import eagerpy as ep
attack = FGSM()
# model_fixed = model_fixed.to(torch.device('cpu')).eval()
epsilons = [0.0,0.0002,0.0005,0.001,0.002,0.01,0.1,0.3,0.5,1.0]
# preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
images, labels = ep.astensors(all_imgs,all_labels)
fmodel = PyTorchModel(model_fixed, bounds=(0, 1), device = 'cpu')
# images, labels = ep.astensors(*samples(fmodel, dataset='cifar10', batchsize=5000))
# images, labels = ep.astensors(*samples(fmodel, dataset="Cifar-10", batchsize=128))
clean_acc = accuracy(fmodel, images, labels)
print(f"clean accuracy:  {clean_acc * 100:.1f} %")

raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)

/home/iailab36/aliakbar/env/lib/python3.10/site-packages/foolbox/models/pytorch.py:36: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  warnings.warn(


clean accuracy:  79.4 %


In [64]:
# clean_acc = accuracy(fmodel, input_2, label_2)
print(f"clean accuracy:  {clean_acc * 100:.1f} %")
robust_accuracy = 1 - success.float32().mean(axis=-1)
print("robust accuracy for perturbations with")
for eps, acc in zip(epsilons, robust_accuracy):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

clean accuracy:  74.3 %
robust accuracy for perturbations with
  Linf norm ≤ 0.0   : 74.1 %
  Linf norm ≤ 0.0002: 71.4 %
  Linf norm ≤ 0.0005: 68.0 %
  Linf norm ≤ 0.0008: 65.8 %
  Linf norm ≤ 0.001 : 63.9 %
  Linf norm ≤ 0.0015: 62.1 %
  Linf norm ≤ 0.002 : 59.7 %
  Linf norm ≤ 0.003 : 55.0 %
  Linf norm ≤ 0.01  : 37.1 %
  Linf norm ≤ 0.1   : 20.3 %
  Linf norm ≤ 0.3   : 16.1 %
  Linf norm ≤ 0.5   : 14.3 %
  Linf norm ≤ 1.0   :  9.0 %


In [37]:
torch.set_grad_enabled(True)
# model = model.to(torch.device('cpu')).eval()
# images, labels = ep.astensors(*samples(fmodel, dataset='cifar10', batchsize=5000))
images, labels = ep.astensors(all_imgs,all_labels)
fmodel_rnn = PyTorchModel(model, bounds=(0, 1), device = 'cpu')
clean_acc = accuracy(fmodel_rnn, images, labels)

print(f"clean accuracy:  {clean_acc * 100:.1f} %")
epsilons = [0.0,0.0002,0.0005,0.001,0.002,0.01,0.1,0.3,0.5,1.0]
raw_advs2, clipped_advs2, success2 = attack(fmodel_rnn, images, labels, epsilons=epsilons)

/home/iailab36/aliakbar/env/lib/python3.10/site-packages/foolbox/models/pytorch.py:36: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  warnings.warn(


clean accuracy:  80.2 %


In [38]:
# clean_acc = accuracy(fmodel, input_2, label_2)
print(f"clean accuracy RNN:  {clean_acc * 100:.1f} %")
robust_accuracy = 1 - success2.float32().mean(axis=-1)
print("robust accuracy for perturbations with")
for eps, acc in zip(epsilons, robust_accuracy):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

clean accuracy RNN:  80.2 %
robust accuracy for perturbations with
  Linf norm ≤ 0.0   : 79.8 %
  Linf norm ≤ 0.0002: 78.3 %
  Linf norm ≤ 0.0005: 75.7 %
  Linf norm ≤ 0.001 : 71.7 %
  Linf norm ≤ 0.002 : 63.8 %
  Linf norm ≤ 0.01  : 24.4 %
  Linf norm ≤ 0.1   :  2.1 %
  Linf norm ≤ 0.3   :  3.7 %
  Linf norm ≤ 0.5   :  5.3 %
  Linf norm ≤ 1.0   :  6.3 %


In [68]:

recs_2 = all_f_recs.nonzero()
recs_1 = (all_f_recs==0).nonzero()
input_2 = all_imgs[recs_2].reshape([recs_2.size()[0],3,32,32])
input_1 = all_imgs[recs_1].reshape([recs_1.size()[0],3,32,32])
label_2 = all_labels[recs_2].reshape([recs_2.size()[0]])
label_1 = all_labels[recs_1].reshape([recs_1.size()[0]])


raw_advs, clipped_advs, success = attack(fmodel, input_1, label_1, epsilons=epsilons)

In [70]:
clean_acc = accuracy(fmodel, input_1, label_1)
print(f"clean accuracy:  {clean_acc * 100:.1f} %")
robust_accuracy = 1 - success.float().mean(axis=-1)
print("robust accuracy for perturbations with")
for eps, acc in zip(epsilons, robust_accuracy):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

clean accuracy:  88.6 %
robust accuracy for perturbations with
  Linf norm ≤ 0.0   : 88.4 %
  Linf norm ≤ 0.0002: 87.1 %
  Linf norm ≤ 0.0005: 85.3 %
  Linf norm ≤ 0.0008: 83.7 %
  Linf norm ≤ 0.001 : 81.3 %
  Linf norm ≤ 0.0015: 78.2 %
  Linf norm ≤ 0.002 : 74.1 %
  Linf norm ≤ 0.003 : 67.7 %
  Linf norm ≤ 0.01  : 50.7 %
  Linf norm ≤ 0.1   : 37.1 %
  Linf norm ≤ 0.3   : 30.1 %
  Linf norm ≤ 0.5   : 24.1 %
  Linf norm ≤ 1.0   : 19.3 %


In [71]:
raw_advs2, clipped_advs2, success2 = attack(fmodel, input_2, label_2, epsilons=epsilons)

In [72]:
clean_acc = accuracy(fmodel, input_2, label_2)
print(f"clean accuracy:  {clean_acc * 100:.1f} %")
robust_accuracy = 1 - success2.float().mean(axis=-1)
print("robust accuracy for perturbations with")
for eps, acc in zip(epsilons, robust_accuracy):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

clean accuracy:  53.4 %
robust accuracy for perturbations with
  Linf norm ≤ 0.0   : 53.7 %
  Linf norm ≤ 0.0002: 51.1 %
  Linf norm ≤ 0.0005: 46.9 %
  Linf norm ≤ 0.0008: 45.1 %
  Linf norm ≤ 0.001 : 43.0 %
  Linf norm ≤ 0.0015: 39.6 %
  Linf norm ≤ 0.002 : 36.1 %
  Linf norm ≤ 0.003 : 31.5 %
  Linf norm ≤ 0.01  : 16.8 %
  Linf norm ≤ 0.1   :  6.0 %
  Linf norm ≤ 0.3   :  6.3 %
  Linf norm ≤ 0.5   :  6.6 %
  Linf norm ≤ 1.0   :  7.3 %


In [74]:
print(label_1.size())
print(label_2.size())

torch.Size([3099])
torch.Size([1901])


In [ ]:
from foolbox import plot
len(clipped_advs[9][0][0])
plot.images(clipped_advs[7][0:5],figsize=[7,7])

In [34]:
import torch

import torchvision.models as models
# model = models.resnet18(pretrained=True).eval().cuda()
# for batch in loader.valid:
#     labels = batch[1]
#     input= batch[0]
#     break
torch.set_grad_enabled(True)
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import FGSM
import eagerpy as ep
attack = FGSM()
# model_fixed = model_fixed.to(torch.device('cpu')).eval()
# epsilons = [0.0,0.0002,0.0005,0.0008,0.001,0.0015,0.002,0.003,0.01,0.1,0.3,0.5,1.0,]
# preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)

fmodel = PyTorchModel(model_mlp, bounds=(0, 1), device = 'cpu')
# images, labels = ep.astensors(*samples(fmodel, dataset='cifar10', batchsize=5000))
images, labels = ep.astensors(all_imgs,all_labels)
# images, labels = ep.astensors(*samples(fmodel, dataset="Cifar-10", batchsize=128))
clean_acc = accuracy(fmodel, images, labels)
print(f"clean accuracy:  {clean_acc * 100:.1f} %")

raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)

/home/iailab36/aliakbar/env/lib/python3.10/site-packages/foolbox/models/pytorch.py:36: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  warnings.warn(


clean accuracy:  79.0 %


In [35]:
# clean_acc = accuracy(fmodel, input_2, label_2)
print(f"clean accuracy MLP:  {clean_acc * 100:.1f} %")
robust_accuracy = 1 - success.float32().mean(axis=-1)
print("robust accuracy for perturbations with")
for eps, acc in zip(epsilons, robust_accuracy):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

clean accuracy MLP:  79.0 %
robust accuracy for perturbations with
  Linf norm ≤ 0.0   : 79.1 %
  Linf norm ≤ 0.0002: 77.1 %
  Linf norm ≤ 0.0005: 74.1 %
  Linf norm ≤ 0.001 : 70.3 %
  Linf norm ≤ 0.002 : 61.4 %
  Linf norm ≤ 0.01  : 29.2 %
  Linf norm ≤ 0.1   :  5.5 %
  Linf norm ≤ 0.3   :  3.8 %
  Linf norm ≤ 0.5   :  4.2 %
  Linf norm ≤ 1.0   :  5.3 %
